In [ ]:
!pip install langchain openai pandas
!pip install langchain[llms]
!pip install -U langchain-community


In [ ]:
import os
from getpass import getpass
OPENAI_API_KEY = getpass("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

OPENAI_API_KEY··········


In [ ]:
import pandas as pd
from langchain.evaluation.qa import CotQAEvalChain
from langchain.chat_models import ChatOpenAI


In [ ]:
# Upload CSV File
from google.colab import files
uploaded = files.upload()  # Manually select your CSV file

# Get the filename
csv_filename = list(uploaded.keys())[0]


Saving CSV-TRAG.csv to CSV-TRAG.csv


In [ ]:
# Load CSV into a DataFrame
# Read the CSV with a fallback encoding (ISO-8859-1 or Windows-1252)
df = pd.read_csv("CSV-TRAG.csv", encoding="Windows-1252")

# Save it back as UTF-8
df.to_csv("CSV-TRAG-UTF8.csv", index=False, encoding="utf-8")

In [ ]:
required_columns = {"Prompt", "Ground Truth", "TRAG"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"CSV file must have columns: {required_columns}")

In [ ]:
# Initialize the language model (e.g., OpenAI's GPT-4)
llm = ChatOpenAI(model_name="gpt-4")

# Initialize the CotQAEvalChain
eval_chain = CotQAEvalChain.from_llm(llm=llm)

In [ ]:
# Prepare examples and predictions
examples = df.to_dict('records')
predictions = [{'result': row['TRAG']} for _, row in df.iterrows()]

In [ ]:
# Evaluate the LLM's answers
results = eval_chain.evaluate(
    examples=examples,
    predictions=predictions,
    question_key='Prompt',
    context_key='Ground Truth',
    prediction_key='result'
)

In [ ]:
# Extract Grade for Text chain in Results

import re

def extract_grade(text):
    """Extracts the grade (CORRECT, INCORRECT, PARTIALLY CORRECT) from evaluation text."""
    match = re.search(r'GRADE:\s*(CORRECT|INCORRECT|PARTIALLY CORRECT)', text)
    return match.group(1) if match else "UNKNOWN"

df['evaluation_text'] = [result.get('text', "No reasoning provided") for result in results]
df['cotqa_grade'] = df['evaluation_text'].apply(extract_grade)

# Display Evaluation Results
display(df.head(5))  # Show only the first 10 rows to preview results

# Save Results to CSV
df.to_csv("evaluation_results.csv", index=False)
print("Evaluation results saved as 'evaluation_results.csv'.")

,Prompt,Ground Truth,TRAG,evaluation_text,cotqa_grade
0,How many blood PK samples will be drawn per st...,29,Each study subject will have a total of 29 ven...,"The context provides the number 29, which corr...",CORRECT
1,Which third party vendors are being used in th...,"Austin Central Lab, Salus IRB, Medidata, Inoti...",The third-party vendors being used in the stud...,The student has correctly identified all third...,CORRECT
2,What is the main risk about urine PK for this ...,All urine must be collected during the study i...,The main risk associated with urine pharmacoki...,The student's answer is comprehensive and fact...,CORRECT
3,How often will the study team meet?\n,"Weekly during start-up and conduct, biweekly d...",The study team will meet with the following fr...,The student's answer matches the context in th...,INCORRECT
4,How many Case Report Forms will be needed for ...,Approximately 77 CRFs per subject will be coll...,"Given the structure of the study, each subject...",The context provided states that approximately...,INCORRECT


Evaluation results saved as 'evaluation_results.csv'.


In [ ]:
aggregation = df.groupby('cotqa_grade')
aggregation['cotqa_grade'].count()

,cotqa_grade
cotqa_grade,
CORRECT,39
INCORRECT,11
